In [1]:
from bs4 import BeautifulSoup
import os
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from flask import Flask

In [2]:
def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)


def scrape():
    
    browser = init_browser()
    mars_collection = {}
    url1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url1)
    html = browser.html
    response = requests.get(url1)
    soup1 = BeautifulSoup(response.text, 'html.parser')
    
    mars_collection["news_title"] = soup1.find('div', class_='content_title').text.strip()
    mars_collection["news_p"] = soup1.find('div', class_='rollover_description_inner').text.strip()
    
    url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url2)
    browser.click_link_by_id('full_image')
    time.sleep(2)
    html = browser.html
    img_soup = BeautifulSoup(html, "html.parser")
    img_url = img_soup.find('img', class_='fancybox-image')['src']
    mars_collection["featured_img_url"] = "https://www.jpl.nasa.gov" + img_url
    
    url3 = 'https://twitter.com/marswxreport?lang=en'
    response = requests.get(url3)
    soup2 = BeautifulSoup(response.text, 'html.parser')
    mars_collection["mars_weather"] = soup2.find('div', class_='js-tweet-text-container').text.strip()
    
    url4 = 'https://space-facts.com/mars/'
    tables = pd.read_html(url4)
    df = tables[0]
    df.columns = ['Mars-Earth_Comparison', 'Mars', 'Earth']
    html_table1 = df.to_html()
    html_table2 = html_table1.replace('\n', '')
    mars_collection["fact_table"] = html_table1
    
    hemisphere_img_urls = []
    
    url4 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
    response = requests.get(url4)
    soup3 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['cerebrus_img'] = soup3.find('img', class_='wide-image')['src']
    mars_collection['cerebrus_title'] = soup3.find('h2', class_='title').text.strip()

    
    url5 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'
    response = requests.get(url5)
    soup4 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['schiaparelli_img'] = soup4.find('img', class_='wide-image')['src']
    mars_collection['schiaparelli_title'] = soup4.find('h2', class_='title').text.strip()
    
    url6 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'
    response = requests.get(url6)    
    soup5 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['syrtis_major_img'] = soup5.find('img', class_='wide-image')['src']
    mars_collection['syrtis_major_title'] = soup5.find('h2', class_='title').text.strip()

    
    url7 = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'
    response = requests.get(url7)
    soup6 = BeautifulSoup(response.text, 'html.parser')
    mars_collection['valles_marineris_img'] = soup6.find('img', class_='wide-image')['src']
    mars_collection['valles_marineris_title'] = soup6.find('h2', class_='title').text.strip()


    return mars_collection    

In [3]:
scrape()

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'news_p': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.",
 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17550_ip.jpg',
 'mars_weather': 'InSight sol 285 (2019-09-15) low -100.8ºC (-149.5ºF) high -27.1ºC (-16.8ºF)\nwinds from the SE at 5.4 m/s (12.2 mph) gusting to 21.3 m/s (47.6 mph)\npressure at 7.40 hPapic.twitter.com/HtCsngFytC',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars-Earth_Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>2

In [4]:
import pymongo

In [5]:
client = pymongo.MongoClient()

In [6]:
mars_scrape = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["mars_db"]


In [7]:
mydb

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_db')

In [8]:
mycol = mydb["mars_data"]
yy=scrape()

In [9]:
print(yy)

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover', 'news_p': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17550_ip.jpg', 'mars_weather': 'InSight sol 285 (2019-09-15) low -100.8ºC (-149.5ºF) high -27.1ºC (-16.8ºF)\nwinds from the SE at 5.4 m/s (12.2 mph) gusting to 21.3 m/s (47.6 mph)\npressure at 7.40 hPapic.twitter.com/HtCsngFytC', 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars-Earth_Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>2</th

In [10]:
x = mycol.insert_one(yy)
x

In [21]:
xx = mycol.find_one()
xx

{'_id': ObjectId('5d803c80fb3e68fc39c8cd68'),
 'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'news_p': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.",
 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17550_ip.jpg',
 'mars_weather': 'InSight sol 285 (2019-09-15) low -100.8ºC (-149.5ºF) high -27.1ºC (-16.8ºF)\nwinds from the SE at 5.4 m/s (12.2 mph) gusting to 21.3 m/s (47.6 mph)\npressure at 7.40 hPapic.twitter.com/HtCsngFytC',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars-Earth_Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 1